## How TomTom represents traffic (brief)

TomTom aggregates probe data (floating car / GPS reports from connected vehicles and apps) and station/sensor data to estimate traffic on road segments. The typical fields you will see in their `flowSegmentData` style responses and our CSV/JSON exports are:
- `currentSpeed`: estimated current speed on the sampled point/segment (km/h).
- `freeFlowSpeed`: the typical uncongested speed for the segment (km/h).
- `travelTime`: estimated travel time for the segment (seconds).
- `confidence` / `level`: a reliability indicator for the estimate.
- `length` / `segmentLength`: segment length used to compute travel time / speed.

TomTom computes traffic by comparing observed speeds to free-flow speeds, applying smoothing, filtering outliers, and using model-based fusion across probes and infrastructure to produce segment-level estimates and confidence scores. For prediction we typically use `currentSpeed` (or `travelTime`) as the target, and time-of-day, day-of-week, weather or nearby segment speeds as features.

In [117]:
# Imports and settings
import json
from pathlib import Path
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

pd.options.display.max_columns = 200

In [118]:
# Helper: load historical JSON dumps from data/tomtom/<region>/
import re

def load_tomtom_jsons(region: str, data_root: Path = Path('..') / 'data' / 'tomtom') -> pd.DataFrame:
    folder = data_root / region
    if not folder.exists():
        raise FileNotFoundError(f'Folder not found: {folder}')
    files = sorted(glob.glob(str(folder / '*.json')))
    if not files:
        raise FileNotFoundError(f'No JSON files found in {folder}')

    rows = []
    for fp in files:
        try:
            j = json.load(open(fp, 'r', encoding='utf-8'))
        except Exception:
            # skip broken files
            continue

        # attempt to parse timestamp from filename, e.g. eindhoven_20251207T205959Z.json
        file_ts = None
        try:
            stem = Path(fp).stem
            m = re.search(r'(\d{8}T\d{6}Z)', stem)
            if m:
                file_ts = pd.to_datetime(m.group(1), format='%Y%m%dT%H%M%SZ', utc=True, errors='coerce')
        except Exception:
            file_ts = None

        # expect a list of records or a top-level object with 'records'/'items'
        if isinstance(j, dict):
            candidates = None
            for k in ('records', 'items', 'features', 'results'):
                if k in j and isinstance(j[k], list):
                    candidates = j[k]
                    break
            if candidates is None:
                candidates = [j]
        elif isinstance(j, list):
            candidates = j
        else:
            continue

        # Heuristic: skip files that contain mostly API error messages and no useful traffic fields
        try:
            n = len(candidates)
            n_error = 0
            n_useful = 0
            for rec in candidates:
                if isinstance(rec, dict):
                    if any(k in rec for k in ('currentSpeed', 'speed', 'travelTime', 'freeFlowSpeed', 'confidence', 'level')):
                        n_useful += 1
                    # look for common error indicators in values
                    if 'error' in rec:
                        n_error += 1
                    else:
                        for v in rec.values():
                            try:
                                vs = str(v).lower()
                                if 'forbidden' in vs or 'client error' in vs or '403' in vs or 'error' in vs or 'http' in vs:
                                    n_error += 1
                                    break
                            except Exception:
                                continue
                else:
                    s = str(rec).lower()
                    if 'forbidden' in s or 'client error' in s or '403' in s or 'error' in s or 'http' in s:
                        n_error += 1
            if n_useful == 0 and n > 0 and (n_error / n) > 0.4:
                continue
        except Exception:
            pass

        for rec in candidates:
            if isinstance(rec, dict):
                rec_copy = dict(rec)
                rec_copy['__file'] = fp
                rec_copy['__file_ts'] = file_ts
                rows.append(rec_copy)
            else:
                rows.append({'value': rec, '__file': fp, '__file_ts': file_ts})

    if not rows:
        return pd.DataFrame()

    df = pd.json_normalize(rows)

    # Prefer explicit time-like columns; convert to datetime safely
    # Note: avoid interpreting numeric duration fields (e.g., 'travelTime') as timestamps
    time_cols = [c for c in df.columns if 'time' in c.lower() or 'date' in c.lower() or c.lower().endswith('timestamp')]
    ts_col = None
    for c in time_cols:
        try:
            s = pd.to_datetime(df[c], utc=True, errors='coerce')
            # guard: accept this column only if a reasonable number of parsed values
            # fall into a plausible year range (e.g., 2000-2100). This prevents fields
            # like 'travelTime' (small integers) becoming 1970-01-01 when coerced.
            if s.notna().any():
                try:
                    good = s.dt.year.between(2000, 2100).sum()
                except Exception:
                    good = 0
                threshold = max(1, int(0.01 * len(s)))
                if good >= threshold:
                    ts_col = c
                    df[c] = s
                    df['_ts'] = df[c]
                    break
        except Exception:
            continue

    # If no explicit timestamp, but filename carried a timestamp, use that
    if ('_ts' not in df.columns or df['_ts'].isna().all()) and '__file_ts' in df.columns:
        try:
            df['_ts'] = pd.to_datetime(df['__file_ts'], utc=True, errors='coerce')
        except Exception:
            df['_ts'] = pd.NaT

    # Do NOT create epoch-based garbage. If still missing, set to NaT and let downstream handle it.
    if '_ts' not in df.columns:
        df['_ts'] = pd.NaT

    # coerce numeric fields
    for nm in ('currentSpeed','freeFlowSpeed','travelTime','speed'):
        if nm in df.columns:
            df[nm] = pd.to_numeric(df[nm], errors='coerce')

    return df


In [119]:
# Example: load historical data for a region
region = 'eindhoven'  # change to your region folder name (slugified)
hist = load_tomtom_jsons(region)
print('Loaded rows:', len(hist))
hist.columns[:80]

Loaded rows: 19266


Index(['lat', 'lon', 'currentSpeed', 'freeFlowSpeed', 'confidence',
       'currentTravelTime', 'freeFlowTravelTime', '__file', '__file_ts',
       'error', '_ts'],
      dtype='object')

In [120]:
# Fetch Meteostat for Eindhoven if not already available for the traffic time range
from pathlib import Path
import sys

# Eindhoven representative coords (change if needed)
lat, lon = 51.4416, 5.4697

# determine start/end dates from the loaded TomTom `hist` dataframe
start = None
end = None
if 'hist' in globals() and isinstance(hist, pd.DataFrame) and not hist.empty:
    # prefer a parsed _ts column, fall back to '__file_ts'
    if '_ts' in hist.columns and hist['_ts'].notna().any():
        s = pd.to_datetime(hist['_ts'], utc=True, errors='coerce').dropna()
    elif '__file_ts' in hist.columns:
        s = pd.to_datetime(hist['__file_ts'], utc=True, errors='coerce').dropna()
    else:
        s = pd.Series(dtype='datetime64[ns]')

    if not s.empty:
        start = s.min().date().isoformat()
        end = s.max().date().isoformat()
else:
    print('No `hist` TomTom data available in this notebook; skipping Meteostat fetch.')

if start is not None and end is not None:
    repo_root = Path.cwd().parent   # adjust if notebook is elsewhere
    out_dir = repo_root / "data" / 'meteostat'
    out_dir.mkdir(exist_ok=True)
    
    fname = f"meteostat_lat{lat}_lon{lon}_{start}_{end}.csv"
    out_fp = out_dir / fname
    if out_fp.exists():
        print(f'Meteostat data already exists for this range: {out_fp} — skipping fetch.')
    else:
        print(f'Fetching Meteostat data for {lat},{lon} from {start} to {end} ...')
        try:
            # ensure repo root is importable so we can import scripts.fetch_meteostat
            if str(repo_root) not in sys.path:
                sys.path.insert(0, str(repo_root))
            import inspect, traceback, importlib, scripts.fetch_meteostat as fm
            saved = fm.fetch_and_save(lat, lon, None, start, end, out_dir=out_dir)
            print('Meteostat fetch completed, saved to', saved)
        except Exception as e:
            print('Could not fetch Meteostat data:', repr(e))
            traceback.print_exc()
else:
    print('Could not determine time range from `hist`; skipping Meteostat fetch.')

Meteostat data already exists for this range: d:\OneDrive - TU Eindhoven\IGNITE\dyson-traffic\data\meteostat\meteostat_lat51.4416_lon5.4697_2025-12-02_2025-12-09.csv — skipping fetch.


In [121]:
# Load Meteostat CSV (if present) and merge weather features into `df_feat`
from pathlib import Path

repo_root = Path.cwd().parent   # adjust if notebook is elsewhere
met_dir = repo_root / "data" / 'meteostat'
met_df = None
if met_dir.exists():
    pattern = f"meteostat_lat{lat}_lon{lon}_{start}_{end}.csv"
    matches = sorted(met_dir.glob(pattern))
    if matches:
        met_fp = matches[-1]
        print('Loading Meteostat from', met_fp)
        met_df = pd.read_csv(met_fp, index_col=0, parse_dates=True)
        # normalize column names to lowercase
        met_df.columns = [c.lower() for c in met_df.columns]
        # improved mapping: recognize Meteostat canonical names and common variants
        colmap = {}
        for c in met_df.columns:
            # Meteostat names: temp, prcp, wspd, wpgt, pres, tsun, coco, etc.
            if 'temp' in c or c in ('t','tavg','tmin','tmax'):
                colmap[c] = 'met_temp'
            elif 'prcp' in c or 'precip' in c or 'rain' in c:
                colmap[c] = 'met_precip'
            elif c in ('wspd','wsp','wind') or ('wind' in c and 'gust' not in c):
                colmap[c] = 'met_wind'
            elif c in ('wpgt','windgust') or 'gust' in c:
                colmap[c] = 'met_wind_gust'
            elif 'cloud' in c:
                colmap[c] = 'met_cloudcover'
            elif c in ('pres','pressure') or 'pres' in c:
                colmap[c] = 'met_pressure'
            elif c in ('tsun','sun'):
                colmap[c] = 'met_tsun'
            elif c in ('coco','condition'):
                colmap[c] = 'met_condition'
        met_df = met_df.rename(columns=colmap)
        # diagnostic: print available columns after mapping
        print('Meteostat columns (after mapping):', list(met_df.columns))
        # ensure datetime index and UTC
        try:
            met_df.index = pd.to_datetime(met_df.index, utc=True, errors='coerce')
        except Exception:
            met_df.index = pd.to_datetime(met_df.index, errors='coerce')
        # compute rolling aggregates where available
        if 'met_precip' in met_df.columns:
            met_df['met_precip_3h'] = met_df['met_precip'].rolling(3, min_periods=1).sum()
            met_df['met_precip_6h'] = met_df['met_precip'].rolling(6, min_periods=1).sum()
            met_df['met_wet'] = met_df['met_precip'] > 0.1
        # ensure hourly index (floor to hour) to be robust
        # specify numeric_only=True to avoid pandas deprecation warnings about mean on non-numeric columns
        met_df = met_df.groupby(met_df.index.floor('h')).mean(numeric_only=True)
    else:
        print('No Meteostat CSV found for pattern', pattern)
else:
    print('Meteostat data directory not found:', met_dir)

# Merge into df_feat if available
# Use globals().get to avoid static/NameError warnings when `df_feat` isn't defined in this kernel/session
df_feat_obj = globals().get('df_feat')
if met_df is not None and isinstance(df_feat_obj, pd.DataFrame) and not df_feat_obj.empty:
    df_feat = df_feat_obj.copy()
    # floor to hourly timestamps for the merge
    df_feat['_ts_hour'] = df_feat['_ts'].dt.floor('h')
    # Select a reasonable set of weather cols if present
    want = [c for c in met_df.columns if c.startswith('met_')]
    # remove duplicates and keep order
    want = list(dict.fromkeys(want))
    if want:
        met_sel = met_df[want]
        # If df_feat already contains any of these columns, drop them first to avoid merge conflicts
        overlap = [c for c in met_sel.columns if c in df_feat.columns]
        if overlap:
            print('Dropping existing weather columns from df_feat to avoid merge conflicts:', overlap)
            df_feat = df_feat.drop(columns=overlap)
        # merge on the floored hour index
        df_feat = df_feat.merge(met_sel, left_on='_ts_hour', right_index=True, how='left')
        # basic missing data handling: forward-fill per pkey where possible
        for col in want:
            if col in df_feat.columns:
                if '_pkey' in df_feat.columns:
                    try:
                        df_feat[col] = df_feat.groupby('_pkey')[col].ffill()
                    except Exception:
                        df_feat[col] = df_feat[col].fillna(method='ffill')
                else:
                    df_feat[col] = df_feat[col].fillna(method='ffill')
                # final fallback fill
                if df_feat[col].dtype == 'bool':
                    df_feat[col] = df_feat[col].fillna(False)
                else:
                    df_feat[col] = df_feat[col].fillna(-1)
        print('Merged Meteostat features into df_feat:', want)
    else:
        print('No recognizable meteostat columns to merge.')
else:
    print('Skipping Meteostat merge (no met_df or no df_feat).')

Loading Meteostat from d:\OneDrive - TU Eindhoven\IGNITE\dyson-traffic\data\meteostat\meteostat_lat51.4416_lon5.4697_2025-12-02_2025-12-09.csv
Meteostat columns (after mapping): ['met_temp', 'dwpt', 'rhum', 'met_precip', 'snow', 'wdir', 'met_wind', 'met_wind_gust', 'met_pressure', 'met_tsun', 'met_condition']
Merged Meteostat features into df_feat: ['met_temp', 'met_precip', 'met_wind', 'met_wind_gust', 'met_pressure', 'met_tsun', 'met_condition', 'met_precip_3h', 'met_precip_6h', 'met_wet']


In [122]:
# Feature engineering helper
def prepare_features(df: pd.DataFrame, timestamp_col_candidates=None) -> pd.DataFrame:
    df = df.copy()

    # If a valid _ts column already exists, ensure it's datetime
    if '_ts' in df.columns:
        try:
            df['_ts'] = pd.to_datetime(df['_ts'], utc=True, errors='coerce')
        except Exception:
            df['_ts'] = pd.NaT

    # If no _ts or all NaT, try other timestamp-like columns
    if df.get('_ts').isna().all():
        ts_cols = timestamp_col_candidates or [c for c in df.columns if 'time' in c.lower() or 'date' in c.lower() or c.lower()=='timestamp']
        for c in ts_cols:
            if c in df.columns:
                try:
                    cand = pd.to_datetime(df[c], utc=True, errors='coerce')
                    # guard: ensure parsed datetimes look like real dates
                    if cand.notna().any():
                        try:
                            good = cand.dt.year.between(2000, 2100).sum()
                        except Exception:
                            good = 0
                        threshold = max(1, int(0.01 * len(cand)))
                        if good >= threshold:
                            df['_ts'] = cand
                            break
                except Exception:
                    continue

    # If still missing, try filename timestamp column '__file_ts'
    if df.get('_ts').isna().all() and '__file_ts' in df.columns:
        try:
            df['_ts'] = pd.to_datetime(df['__file_ts'], utc=True, errors='coerce')
        except Exception:
            df['_ts'] = pd.NaT

    # As last resort create a dummy increasing time index (2000-01-01) to avoid 1970 epoch
    if df.get('_ts').isna().all():
        df['_ts'] = pd.date_range('2000-01-01', periods=len(df), freq='T', tz='UTC')

    df['_hour'] = df['_ts'].dt.hour
    df['_dow'] = df['_ts'].dt.dayofweek
    # cyclic encoding for hour
    df['hour_sin'] = np.sin(2 * np.pi * df['_hour'] / 24.0)
    df['hour_cos'] = np.cos(2 * np.pi * df['_hour'] / 24.0)

    # spatial grouping: round lat/lon if available to group nearby samples
    lat_cols = [c for c in df.columns if c.lower().endswith('lat') or c.lower().startswith('lat')]
    lon_cols = [c for c in df.columns if c.lower().endswith('lon') or c.lower().startswith('lon')]
    if lat_cols and lon_cols:
        latc = lat_cols[0]; lonc = lon_cols[0]
        df['_plat'] = pd.to_numeric(df[latc], errors='coerce').round(5)
        df['_plon'] = pd.to_numeric(df[lonc], errors='coerce').round(5)
        df['_pkey'] = df['_plat'].astype(str) + '_' + df['_plon'].astype(str)
    else:
        df['_pkey'] = 'no_spatial'

    # target: prefer currentSpeed, fallback to speed or travelTime/length if available
    if 'currentSpeed' in df.columns:
        df['target_speed'] = pd.to_numeric(df['currentSpeed'], errors='coerce')
    elif 'speed' in df.columns:
        df['target_speed'] = pd.to_numeric(df['speed'], errors='coerce')
    elif 'travelTime' in df.columns and 'length' in df.columns:
        df['target_speed'] = pd.to_numeric(df['length'], errors='coerce') / pd.to_numeric(df['travelTime'], errors='coerce') * 3.6
    else:
        df['target_speed'] = np.nan

    # sort and create simple lag features per pkey
    df = df.sort_values('_ts')
    df['lag1'] = df.groupby('_pkey')['target_speed'].shift(1)
    df['lag2'] = df.groupby('_pkey')['target_speed'].shift(2)
    df['rolling_mean_3'] = df.groupby('_pkey')['target_speed'].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)

    # drop rows without target
    df = df[~df['target_speed'].isna()]

    return df


In [123]:
# Prepare dataset and train an improved model (LightGBM/XGBoost fallback, time-aware split, randomized CV)

# build base features list
base_feats = ['hour_sin','hour_cos','_dow','lag1','lag2','rolling_mean_3']
# include base features that exist
feat_cols = [c for c in base_feats if c in df_feat.columns]

# include any Meteostat features we merged (prefixed with met_)
met_feats = [c for c in df_feat.columns if isinstance(c, str) and c.startswith('met_')]
if met_feats:
    print('Including Meteostat features:', met_feats)
    # append met features after base features (avoid duplicates)
    for f in met_feats:
        if f not in feat_cols:
            feat_cols.append(f)

print('Using feature columns:', feat_cols)
X = df_feat[feat_cols].fillna(-1)
y = df_feat['target_speed']

# Basic data checks and defensive behavior
if df_feat.empty or len(X) == 0:
    print('Not enough data after feature engineering to train a model.')
else:
    # Time-based split: train on earlier data, test on the most recent 2 days if possible
    if '_ts' in df_feat.columns and df_feat['_ts'].notna().any():
        cutoff = df_feat['_ts'].max() - pd.Timedelta(days=2)
        train_idx = df_feat['_ts'] <= cutoff
        test_idx = df_feat['_ts'] > cutoff
        if train_idx.sum() < 100 or test_idx.sum() < 20:
            # fallback to a random split if not enough data
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        else:
            X_train, X_test = X.loc[train_idx], X.loc[test_idx]
            y_train, y_test = y.loc[train_idx], y.loc[test_idx]
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print('Training rows:', len(X_train), 'Test rows:', len(X_test))

    # If training set is small, skip expensive CV/hyper-search and use a simple estimator
    MIN_SAMPLES_FOR_SEARCH = 50
    model = None
    if len(X_train) < MIN_SAMPLES_FOR_SEARCH:
        print(f'Only {len(X_train)} training rows (<{MIN_SAMPLES_FOR_SEARCH}). Skipping hyperparameter search and using RandomForest.')
        model = RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)
        model.fit(X_train, y_train)
    else:
        # Try LightGBM first (wrapped so any failure falls back cleanly)
        try:
            import lightgbm as lgb
            from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
            estimator = lgb.LGBMRegressor(objective='regression', n_jobs=-1, random_state=42)
            # choose cv splits conservatively based on data size
            n_splits = min(3, max(2, int(len(X_train) // 20)))
            cv = TimeSeriesSplit(n_splits=n_splits)
            param_dist = {
                'n_estimators': [100, 200, 400],
                'learning_rate': [0.01, 0.05, 0.1],
                'num_leaves': [31, 63, 127],
                'max_depth': [-1, 6, 12],
            }
            rsearch = RandomizedSearchCV(estimator, param_distributions=param_dist, n_iter=12, cv=cv, scoring='neg_mean_absolute_error', n_jobs=1, random_state=42, verbose=1)
            rsearch.fit(X_train, y_train)
            model = rsearch.best_estimator_
            print('Best params (LightGBM):', rsearch.best_params_)
        except Exception as e:
            print('LightGBM not available or failed, fallback to XGBoost/RandomForest:', e)
            try:
                import xgboost as xgb
                from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
                estimator = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=1, random_state=42)
                n_splits = min(3, max(2, int(len(X_train) // 20)))
                cv = TimeSeriesSplit(n_splits=n_splits)
                param_dist = {
                    'n_estimators': [100,200,400],
                    'learning_rate': [0.01,0.05,0.1],
                    'max_depth': [3,6,10],
                }
                rsearch = RandomizedSearchCV(estimator, param_distributions=param_dist, n_iter=12, cv=cv, scoring='neg_mean_absolute_error', n_jobs=1, random_state=42, verbose=1)
                rsearch.fit(X_train, y_train)
                model = rsearch.best_estimator_
                print('Best params (XGBoost):', rsearch.best_params_)
            except Exception as e2:
                print('XGBoost not available or failed, using RandomForest:', e2)
                model = RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)
                model.fit(X_train, y_train)

    # If we used a search object, ensure model is fitted (some branches already fitted)
    try:
        model.predict(X_test[:1])
    except Exception:
        model.fit(X_train, y_train)

    # Predictions and evaluation
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    mse_val = mean_squared_error(y_test, pred)
    rmse_val = np.sqrt(mse_val)
    print('MAE:', mae)
    print('RMSE:', rmse_val)

    # Feature importances if available
    try:
        importances = None
        if hasattr(model, 'feature_importances_'):
            importances = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
        elif hasattr(model, 'booster') and hasattr(model, 'get_score'):
            # xgboost
            try:
                fmap = model.get_booster().get_score(importance_type='weight')
                importances = pd.Series(fmap).reindex(X_train.columns).fillna(0).sort_values(ascending=False)
            except Exception:
                importances = None
        if importances is not None:
            print('Feature importances:')
            print(importances)
    except Exception as e:
        print('Could not extract feature importances:', e)


Including Meteostat features: ['met_temp_x', 'met_precip_x', 'met_wind_x', 'met_wind_gust_x', 'met_pressure_x', 'met_tsun_x', 'met_condition_x', 'met_precip_3h_x', 'met_precip_6h_x', 'met_wet_x', 'met_temp', 'met_precip', 'met_wind', 'met_wind_gust', 'met_pressure', 'met_tsun', 'met_condition', 'met_precip_3h', 'met_precip_6h', 'met_wet']
Using feature columns: ['hour_sin', 'hour_cos', '_dow', 'lag1', 'lag2', 'rolling_mean_3', 'met_temp_x', 'met_precip_x', 'met_wind_x', 'met_wind_gust_x', 'met_pressure_x', 'met_tsun_x', 'met_condition_x', 'met_precip_3h_x', 'met_precip_6h_x', 'met_wet_x', 'met_temp', 'met_precip', 'met_wind', 'met_wind_gust', 'met_pressure', 'met_tsun', 'met_condition', 'met_precip_3h', 'met_precip_6h', 'met_wet']
Training rows: 13961 Test rows: 5304
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000135 seconds.
You can set `force_row_wise=true` to remove the overhead.

In [124]:
# Diagnostics: Clean duplicate Meteostat columns
try:
    met_cols = [c for c in df_feat.columns if isinstance(c, str) and c.startswith('met_')]
    print('Found meteostat columns:', met_cols)
    from collections import defaultdict
    groups = defaultdict(list)
    for c in met_cols:
        base = c[:-2] if (c.endswith('_x') or c.endswith('_y')) else c
        groups[base].append(c)
    keep = []
    for base, lst in groups.items():
        chosen = None
        # prefer _x, then plain, then _y, then first
        for suffix in ('_x', '', '_y'):
            candidate = base + suffix if suffix else base
            if candidate in lst:
                chosen = candidate
                break
        if chosen is None:
            chosen = lst[0]
        keep.append(chosen)
    drop = [c for c in met_cols if c not in keep]
    if drop:
        print('Dropping duplicate weather columns:', drop)
        df_feat.drop(columns=drop, inplace=True, errors='ignore')
    else:
        print('No duplicate weather columns to drop')
    print('Final meteostat features:', [c for c in df_feat.columns if isinstance(c, str) and c.startswith('met_')])
except Exception as e:
    print('Could not clean meteostat columns:', e)

Found meteostat columns: ['met_temp_x', 'met_precip_x', 'met_wind_x', 'met_wind_gust_x', 'met_pressure_x', 'met_tsun_x', 'met_condition_x', 'met_precip_3h_x', 'met_precip_6h_x', 'met_wet_x', 'met_temp', 'met_precip', 'met_wind', 'met_wind_gust', 'met_pressure', 'met_tsun', 'met_condition', 'met_precip_3h', 'met_precip_6h', 'met_wet']
Dropping duplicate weather columns: ['met_temp', 'met_precip', 'met_wind', 'met_wind_gust', 'met_pressure', 'met_tsun', 'met_condition', 'met_precip_3h', 'met_precip_6h', 'met_wet']
Final meteostat features: ['met_temp_x', 'met_precip_x', 'met_wind_x', 'met_wind_gust_x', 'met_pressure_x', 'met_tsun_x', 'met_condition_x', 'met_precip_3h_x', 'met_precip_6h_x', 'met_wet_x']


In [125]:
# Ablation: compare model performance with and without Meteostat features
def evaluate_model(X, y, n_splits=3, use_lgb=True):
    from sklearn.model_selection import TimeSeriesSplit
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np
    tscv = TimeSeriesSplit(n_splits=n_splits)
    maes=[]; rmses=[]
    for train_idx, val_idx in tscv.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        if use_lgb:
            try:
                import lightgbm as lgb
                model = lgb.LGBMRegressor(objective='regression', n_jobs=-1, random_state=42, n_estimators=200)
            except Exception:
                from sklearn.ensemble import RandomForestRegressor
                model = RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)
        else:
            from sklearn.ensemble import RandomForestRegressor
            model = RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)
        model.fit(X_tr, y_tr)
        p = model.predict(X_val)
        maes.append(mean_absolute_error(y_val, p))
        rmses.append(np.sqrt(mean_squared_error(y_val, p)))
    return {'mae': float(np.mean(maes)), 'rmse': float(np.mean(rmses))}

try:
    feat_cols = [c for c in df_feat.columns if c not in ('target_speed','_ts','_ts_hour')]
    X_all = df_feat[feat_cols].fillna(-1)
    y = df_feat['target_speed']
    # features excluding meteostat
    feat_no_met = [c for c in feat_cols if not (isinstance(c, str) and c.startswith('met_'))]
    X_no_met = df_feat[feat_no_met].fillna(-1)
    print('Evaluating with Meteostat features...')
    res_all = evaluate_model(X_all, y, n_splits=min(3, max(2, int(len(X_all)//50))))
    print('With Meteostat -> MAE: {mae:.4f}, RMSE: {rmse:.4f}'.format(**res_all))
    print('Evaluating without Meteostat features...')
    res_no = evaluate_model(X_no_met, y, n_splits=min(3, max(2, int(len(X_no_met)//50))))
    print('Without Meteostat -> MAE: {mae:.4f}, RMSE: {rmse:.4f}'.format(**res_no))
except Exception as e:
    print('Could not run ablation evaluation:', e)

Evaluating with Meteostat features...
Could not run ablation evaluation: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: __file: object, __file_ts: datetime64[ns, UTC], _pkey: object


In [126]:
# Model interpretability: SHAP if available, else permutation importance
try:
    # fit a final model on all data using preferred algorithm
    X = df_feat[[c for c in df_feat.columns if c not in ('target_speed','_ts','_ts_hour')]].fillna(-1)
    y = df_feat['target_speed']
    try:
        import lightgbm as lgb
        model = lgb.LGBMRegressor(objective='regression', n_jobs=-1, random_state=42, n_estimators=400)
    except Exception:
        from sklearn.ensemble import RandomForestRegressor
        model = RandomForestRegressor(n_estimators=400, n_jobs=-1, random_state=42)
    model.fit(X, y)
    # try SHAP
    try:
        import shap
        explainer = shap.Explainer(model, X)
        shap_values = explainer(X)
        print('SHAP summary (top 20):')
        import pandas as pd, numpy as np
        mean_abs = np.abs(shap_values.values).mean(axis=0)
        fi = pd.Series(mean_abs, index=X.columns).sort_values(ascending=False)
        print(fi.head(20))
    except Exception as e_shap:
        print('SHAP not available or failed:', e_shap)
        # fallback to permutation importance
        try:
            from sklearn.inspection import permutation_importance
            res = permutation_importance(model, X, y, n_repeats=10, random_state=42, n_jobs=-1)
            import pandas as pd
            pi = pd.Series(res.importances_mean, index=X.columns).sort_values(ascending=False)
            print('Permutation importance (top 20):')
            print(pi.head(20))
        except Exception as e_pi:
            print('Permutation importance failed:', e_pi)
except Exception as e:
    print('Could not compute interpretability results:', e)

Could not compute interpretability results: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: __file: object, __file_ts: datetime64[ns, UTC], _pkey: object


- This notebook is a starter template. Improvements: spatial joins to canonical road segments, using TomTom segment IDs, adding weather and event features, and training more advanced time-series models (LSTM, temporal convolutional nets).
- For production, collect stable segment identifiers (TomTom segment IDs) and build time-series per segment.
- Consider persistent caching for reverse-geocode and careful rate-limit handling when fetching live data.